<a href="https://colab.research.google.com/github/laresistance-bolivia/BOLIVIA_ELECCIONES_2019/blob/master/ANALISIS_COMPUTO_OFICIAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis de Actas computadas en las elecciones presidenciales 2019 de Bolivia

- De preferencia corre el notebook en Google Colab

- La fuente de datos se descarga de manera automática de la siguiente fuente:
https://drive.google.com/drive/folders/1z-Jwked3bX3ZiHEuy8ckvnSzRwAz62RF

- La mayor parte de los gráficos son interactivos, puedes habilitar y deshabilitar opciones desde el menú lateral y hacer zoom sobre los mismos.



In [1]:
# Download DB
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1StCAGlX-ddYqFcdmtQWDUCuRRqKrfGgL' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1StCAGlX-ddYqFcdmtQWDUCuRRqKrfGgL" -O historico_actas.zip && rm -rf /tmp/cookies.txt

--2019-10-27 19:35:45--  https://docs.google.com/uc?export=download&confirm=Bj7k&id=1StCAGlX-ddYqFcdmtQWDUCuRRqKrfGgL
Resolving docs.google.com (docs.google.com)... 108.177.111.102, 108.177.111.100, 108.177.111.113, ...
Connecting to docs.google.com (docs.google.com)|108.177.111.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-2k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/9i9kj35ltiukvkgk12lt4mb59ovtq17o/1572199200000/08171737692115128905/*/1StCAGlX-ddYqFcdmtQWDUCuRRqKrfGgL?e=download [following]
--2019-10-27 19:35:45--  https://doc-0g-2k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/9i9kj35ltiukvkgk12lt4mb59ovtq17o/1572199200000/08171737692115128905/*/1StCAGlX-ddYqFcdmtQWDUCuRRqKrfGgL?e=download
Resolving doc-0g-2k-docs.googleusercontent.com (doc-0g-2k-docs.googleusercontent.com)... 74.125.126.132, 2607:f8b0:4001:c1d::84
Connecting to doc-0g-2k-docs.googleusercontent

In [2]:
!unzip historico_actas.zip

Archive:  historico_actas.zip
  inflating: DatosActasPorHora.db    


In [4]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.express as px
init_notebook_mode(connected=True)

In [0]:
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

In [0]:
# DataBase connection
conn = sqlite3.connect("DatosActasPorHora.db")

In [0]:
# Table Description
# query = "PRAGMA table_info(ActasPresidenteComputo);"
# pd.read_sql_query(query, conn)

In [9]:
# Registros por archivo
query = """
SELECT FechaHoraArchivo, count(*) as total_registros
FROM ActasPresidenteComputo 
group by FechaHoraArchivo
"""
df_actas_qty = pd.read_sql_query(query, conn)
df_actas_qty['FechaHoraArchivo'] = pd.to_datetime(df_actas_qty['FechaHoraArchivo'])
df_actas_qty

,FechaHoraArchivo,total_registros
0,2019-10-20 22:29:52,707
1,2019-10-20 23:31:51,1565
2,2019-10-21 00:34:51,4031
3,2019-10-21 01:34:51,5115
4,2019-10-21 02:37:51,5642
...,...,...
99,2019-10-25 02:56:27,34551
100,2019-10-25 03:59:34,34551
101,2019-10-25 05:14:34,34551
102,2019-10-25 06:17:34,34551


In [0]:
# df_actas_qty.plot(x="FechaHoraArchivo", y="total_registros")

## Cantidad de Registros en Actas

In [11]:
configure_plotly_browser_state()
data = go.Scatter(
           x=df_actas_qty['FechaHoraArchivo'],
           y=df_actas_qty['total_registros'])

layout = go.Layout(title='Cantidad de Registros en actas', 
                   xaxis=dict(title='Fecha'),
                   yaxis=dict(title='Cantidad'))

fig = go.Figure(data, layout=layout)
iplot(fig)

##### Cálculos

In [0]:
cols_partidos = ['CC', 'FPV', 'MTS', 'UCS', 'MAS', 'M21F', 'PDC', 'MNR', 'PanBol']

In [0]:
# Obtener todas las actas
query = "SELECT * FROM ActasPresidenteComputo"
df_actas_all = pd.read_sql_query(query, conn)
df_actas_all['FechaHoraArchivo'] = pd.to_datetime(df_actas_all['FechaHoraArchivo'])
df_actas_all.sort_values( by=['FechaHoraArchivo'], inplace=True)

In [0]:
# Solo valores numéricos
# df_actas_all_numericos = df_actas_all[['CodigoMesa', 'FechaHoraArchivo' ] + cols_partidos + ['Validos', 'Blancos', 'Nulos', 'Inscritos']]
# df_actas_all_numericos.head()

In [15]:
# Cálculo de votos válidos
df_actas_all['_ValidosCalculado'] = df_actas_all[cols_partidos].sum(axis=1)
df_actas_all['_Validos_menos_ValidosCalculado'] = df_actas_all['Validos'] - df_actas_all['_ValidosCalculado']
df_actas_all['_Validos_menos_ValidosCalculado_Abs'] = df_actas_all['_Validos_menos_ValidosCalculado'].abs()

# Ausentismo
df_actas_all['_Ausentismo_OEP'] = df_actas_all['Inscritos'] - (df_actas_all['Validos'] + df_actas_all['Blancos'] + df_actas_all['Nulos'])
df_actas_all['_Ausentismo_Calculado'] = df_actas_all['Inscritos'] - (df_actas_all['_ValidosCalculado'] + df_actas_all['Blancos'] + df_actas_all['Nulos'])

# Total de Votantes por mesa
df_actas_all['_Votantes'] = df_actas_all['_ValidosCalculado'] + df_actas_all['Blancos'] + df_actas_all['Nulos']

# Asistencia Total en mesas
df_actas_all['_TotalAsistencia'] = 'NO'
df_actas_all.loc[df_actas_all['_Votantes'] == df_actas_all['Inscritos'] , '_TotalAsistencia'] = 'SI' 


# Mesa Ganadora
df_actas_all['_PartidoGanador'] = df_actas_all[cols_partidos].idxmax(axis=1)


df_actas_all.head()

,Id,FechaHoraArchivo,Pais,NumeroDepartamento,Provincia,NumeroMunicipio,Municipio,Circunscripcion,Localidad,Recinto,NumeroMesa,CodigoMesa,Eleccion,Inscritos,CC,FPV,MTS,UCS,MAS,M21F,PDC,MNR,PanBol,Validos,Blancos,Nulos,Computada,Departamento,_ValidosCalculado,_Validos_menos_ValidosCalculado,_Validos_menos_ValidosCalculado_Abs,_Ausentismo_OEP,_Ausentismo_Calculado,_Votantes,_TotalAsistencia,_PartidoGanador
0,5637117,2019-10-20 22:29:52,Bolivia,1,Oropeza,1,Sucre,,Sucre,Salon Multifuncional America,10177,100320127940022503,Presidente y Vicepresidente,232,104,3,1,0,41,9,26,2,3,189,5,9,Computada,Chuquisaca,189,0,0,29,29,203,NO,CC
467,5637584,2019-10-20 22:29:52,Bolivia,5,Tomas Frias,1,Potosí,,Potosí,Unidad Educativa Juan Pablo II,50635,100320514940048414,Presidente y Vicepresidente,228,114,0,0,1,31,15,27,1,3,192,2,10,Computada,Potosí,192,0,0,24,24,204,NO,CC
468,5637585,2019-10-20 22:29:52,Bolivia,5,Tomas Frias,1,Potosí,,Potosí,Unidad Educativa Juan Pablo II,50636,100320514940048415,Presidente y Vicepresidente,232,113,0,2,2,41,11,30,1,1,201,4,11,Computada,Potosí,201,0,0,16,16,216,NO,CC
469,5637586,2019-10-20 22:29:52,Bolivia,5,Tomas Frias,1,Potosí,,Potosí,Unidad Educativa Juan Pablo II,50637,100320514940048416,Presidente y Vicepresidente,227,119,0,1,1,33,15,25,0,2,196,1,9,Computada,Potosí,196,0,0,21,21,206,NO,CC
470,5637587,2019-10-20 22:29:52,Bolivia,5,Tomas Frias,1,Potosí,,Potosí,Unidad Educativa Juan Pablo II,50638,100320514940048417,Presidente y Vicepresidente,238,107,2,5,2,45,11,32,1,2,207,3,8,Computada,Potosí,207,0,0,20,20,218,NO,CC


### Ultimo archivo disponible

In [16]:
# Acta final (En base a la fecha máxima)
acta_final_fecha = str(df_actas_all['FechaHoraArchivo'].max())
acta_final_fecha

'2019-10-25 07:20:35'

In [17]:
# Cálculos sobre la última acta
print("Cálculos sobre la última acta")
# Errores de calculo de sumatoria votos validos  en la ultima acta 
df_validos_mal_computados = df_actas_all[(df_actas_all['FechaHoraArchivo'] == acta_final_fecha) & (df_actas_all['_Validos_menos_ValidosCalculado_Abs'] != 0)].groupby(by=['Departamento', 'Municipio'] ).sum()[['_Validos_menos_ValidosCalculado_Abs', '_Ausentismo_Calculado']].reset_index()

## Ausentismo ultima acta
df_ausentismo_calculado = df_actas_all[(df_actas_all['FechaHoraArchivo'] == acta_final_fecha) & (df_actas_all['_Ausentismo_Calculado'] != 0)].groupby(by=['Departamento', 'Municipio', '_PartidoGanador'] ).sum()[['_Ausentismo_Calculado']].reset_index()

## Resumen 
df_actas_inscritos = df_actas_all[(df_actas_all['FechaHoraArchivo'] == acta_final_fecha)].groupby(by=['Pais', 'Departamento', 'Provincia']).sum()[['Inscritos', '_ValidosCalculado', '_Ausentismo_Calculado', 'MAS', 'CC']].reset_index()
df_actas_inscritos.sample(5)

Cálculos sobre la última acta


,Pais,Departamento,Provincia,Inscritos,_ValidosCalculado,_Ausentismo_Calculado,MAS,CC
90,Bolivia,La Paz,Larecaja,39134,31667,4546,18085,4849
41,Argentina,Santa Cruz - Argentina,Güer Aike,864,702,137,648,19
164,Brasil,Sao Paulo,Sao Paulo,44232,30449,12772,21783,2709
79,Bolivia,Cochabamba,Tiraque,27871,24354,2426,21298,1137
207,Paises Bajos,Holanda,Holanda,133,90,42,18,62


## Error de suma en votos válidos por municipio

In [18]:
configure_plotly_browser_state()
fig = px.bar(df_validos_mal_computados, x="Municipio", y="_Validos_menos_ValidosCalculado_Abs", title="Votos válidos mal sumados")
fig.show()


## Mesas con asistencia total

In [19]:
## Mesas con total asistencia
# Corrigieron esto en la última version de las actas
df_total_asistencia = df_actas_all[(df_actas_all['FechaHoraArchivo'] == acta_final_fecha) & (df_actas_all['_TotalAsistencia'] == 'SI')].groupby(by=['Departamento', 'Provincia', 'Municipio', '_PartidoGanador'] ).sum()[['Inscritos', 'MAS', 'CC']].reset_index()

# df_total_asistencia = df_actas_all[df_actas_all['_TotalAsistencia'] == 'SI'].groupby(by=['FechaHoraArchivo', 'Departamento', 'Provincia', '_PartidoGanador'] ).sum()[['Inscritos', 'MAS', 'CC']].reset_index()
print(len(df_total_asistencia))
print("Muestra de 10 mesas")
df_total_asistencia.sample(10)

132
Muestra de 10 mesas


,Departamento,Provincia,Municipio,_PartidoGanador,Inscritos,MAS,CC
79,Potosí,Chayanta,Ravelo,MAS,23,12,0
53,La Paz,Murillo,Mecapaca,MAS,52,44,0
113,Santa Cruz,Ichilo,San Carlos,CC,42,12,18
123,Santa Cruz,Warnes,Warnes,CC,326,111,130
90,Santa Cruz,Andres Ibañez,Cotoca,CC,230,55,110
92,Santa Cruz,Andres Ibañez,El Torno,MAS,173,117,11
50,La Paz,Murillo,Achocalla,MAS,83,46,11
40,La Paz,Gualberto Villarroel,Papel Pampa,MAS,63,37,8
93,Santa Cruz,Andres Ibañez,La Guardia,CC,38,12,20
2,Beni,General José Ballivián,Puerto Menor de Rurrenabaque,MAS,229,58,40


### Mesas con asistencia total por municipios y partido ganador

In [20]:
configure_plotly_browser_state()
fig = px.bar(df_total_asistencia, x="Municipio", y="Inscritos", title="_Asistencia Total en Mesas", color="_PartidoGanador")
fig.show()


## Inscritos por Departamento

In [21]:
configure_plotly_browser_state()
fig = px.bar(df_actas_inscritos, x="Departamento", y="Inscritos",  title="Inscritos", color= "Pais")
fig.show()

## Ausentismo por departamento

In [22]:
configure_plotly_browser_state()
fig = px.bar(df_actas_inscritos, x="Departamento", y="_Ausentismo_Calculado",  title="Ausentismo", color= "Pais")
fig.show()

## Ausentismo por municipio y partido ganador

In [23]:
configure_plotly_browser_state()
fig = px.bar(df_ausentismo_calculado, x="Municipio", y="_Ausentismo_Calculado", title="Ausentismo", color='_PartidoGanador')
fig.show()

## Votos válidos por departamento

In [24]:
configure_plotly_browser_state()
fig = px.bar(df_actas_inscritos, x="Departamento", y="_ValidosCalculado",  title="Votos Válidos", color= "Pais")
fig.show()

## Votos válidos a favor del MAS

In [25]:
configure_plotly_browser_state()
fig = px.bar(df_actas_inscritos, x="Departamento", y="MAS",  title="Votos Válidos a favor del MAS", color= "Pais")
fig.show()

## Votos válidos a favor de CC

In [26]:
configure_plotly_browser_state()
fig = px.bar(df_actas_inscritos, x="Departamento", y="CC",  title="Votos Válidos a favor de CC", color= "Pais")
fig.show()

## Actas históricas analizadas

In [27]:
print("Total Actas del repositorio histórico")
len(df_actas_all)

Total Actas del repositorio histórico


2832795

In [28]:
df_actas_modificadas = df_actas_all [~df_actas_all.loc[:, ~df_actas_all.columns.isin(['FechaHoraArchivo', 'Id'])].duplicated(keep='first')]
print("Actas únicas con modificaciones")
len(df_actas_modificadas)
# df_actas_all.duplicated(keep='first', subset=[~df_actas_all.columns.isin(['FechaHoraArchivo', 'Id'])])

Actas únicas con modificaciones


34658

In [29]:
# Actas modificadas multiples veces
df_actas_modificadas_multiples_veces = df_actas_modificadas.groupby(by=['Pais', 'Departamento', 'Municipio', 'CodigoMesa']).count()['Inscritos'].reset_index()
df_actas_modificadas_multiples_veces = df_actas_modificadas_multiples_veces[df_actas_modificadas_multiples_veces['Inscritos'] > 1]
print("Actas modificadas más de una vez")
len(df_actas_modificadas_multiples_veces)

Actas modificadas más de una vez


43

In [30]:
print("Actas editadas")
df_actas_modificadas_multiples_veces.sort_values('Inscritos',  ascending=False).head(20)

Actas editadas


,Pais,Departamento,Municipio,CodigoMesa,Inscritos
1277,Bolivia,Beni,Riberalta,100320825402557402,2
31145,Bolivia,Santa Cruz,Santa Cruz de La Sierra,100320722342565509,2
31137,Bolivia,Santa Cruz,Santa Cruz de La Sierra,100320722342565501,2
31138,Bolivia,Santa Cruz,Santa Cruz de La Sierra,100320722342565502,2
31139,Bolivia,Santa Cruz,Santa Cruz de La Sierra,100320722342565503,2
31140,Bolivia,Santa Cruz,Santa Cruz de La Sierra,100320722342565504,2
31141,Bolivia,Santa Cruz,Santa Cruz de La Sierra,100320722342565505,2
31142,Bolivia,Santa Cruz,Santa Cruz de La Sierra,100320722342565506,2
31143,Bolivia,Santa Cruz,Santa Cruz de La Sierra,100320722342565507,2
31144,Bolivia,Santa Cruz,Santa Cruz de La Sierra,100320722342565508,2


In [31]:
df_mesas_corregidas_detalle = pd.DataFrame()
for mesa in df_actas_modificadas_multiples_veces['CodigoMesa']:
  # print(mesa)
  df_mesa = df_actas_all[df_actas_all['CodigoMesa'] == mesa].copy()
  df_mesa['Duplicated'] = df_mesa.duplicated(subset=df_mesa.columns[~df_mesa.columns.isin(['Id',	'FechaHoraArchivo'])])
  df_mesas_corregidas_detalle = df_mesas_corregidas_detalle.append(df_mesa[df_mesa['Duplicated'] == False], ignore_index=True)

df_mesas_corregidas_detalle.head(50)

,Id,FechaHoraArchivo,Pais,NumeroDepartamento,Provincia,NumeroMunicipio,Municipio,Circunscripcion,Localidad,Recinto,NumeroMesa,CodigoMesa,Eleccion,Inscritos,CC,FPV,MTS,UCS,MAS,M21F,PDC,MNR,PanBol,Validos,Blancos,Nulos,Computada,Departamento,_ValidosCalculado,_Validos_menos_ValidosCalculado,_Validos_menos_ValidosCalculado_Abs,_Ausentismo_OEP,_Ausentismo_Calculado,_Votantes,_TotalAsistencia,_PartidoGanador,Duplicated
0,6243637,2019-10-22 13:16:51,Bolivia,8,Vaca Diez,1,Riberalta,,Riberalta,U.E. Litoral,80632,100320825402557402,Presidente y Vicepresidente,239,52,0,1,6,56,25,11,13,0,164,54,4,Computada,Beni,164,0,0,17,17,222,NO,MAS,False
1,6430965,2019-10-22 19:34:51,Bolivia,8,Vaca Diez,1,Riberalta,,Riberalta,U.E. Litoral,80632,100320825402557402,Presidente y Vicepresidente,239,72,1,1,0,66,22,32,9,0,203,12,7,Computada,Beni,203,0,0,17,17,222,NO,CC,False
2,5733799,2019-10-21 14:10:51,Bolivia,1,Oropeza,1,Sucre,,Sucre,Fundacion Pachamama,10291,100320127940050603,Presidente y Vicepresidente,231,140,0,2,0,36,3,17,1,0,199,0,7,Computada,Chuquisaca,199,0,0,25,25,206,NO,CC,False
3,7955036,2019-10-24 19:28:43,Bolivia,1,Oropeza,1,Sucre,,Sucre,Fundacion Pachamama,10291,100320127940050603,Presidente y Vicepresidente,231,139,0,2,0,36,3,17,1,0,199,0,7,Computada,Chuquisaca,198,1,1,25,26,205,NO,CC,False
4,5761579,2019-10-21 16:16:51,Bolivia,1,Oropeza,1,Sucre,,Sucre,IPTK-N. Aguirre 560 / CERPI,10514,100320127940050801,Presidente y Vicepresidente,226,88,0,7,2,31,17,0,13,9,167,29,6,Computada,Chuquisaca,167,0,0,24,24,202,NO,CC,False
5,7714636,2019-10-24 12:07:43,Bolivia,1,Oropeza,1,Sucre,,Sucre,IPTK-N. Aguirre 560 / CERPI,10514,100320127940050801,Presidente y Vicepresidente,226,142,0,2,0,40,3,7,2,0,196,2,4,Computada,Chuquisaca,196,0,0,24,24,202,NO,CC,False
6,6254662,2019-10-22 14:19:51,Bolivia,2,Muñecas,1,Chuma,,Chajlaya,Escuela Chajlaya,26765,100320202420240301,Presidente y Vicepresidente,207,1,2,11,0,102,1,0,0,0,117,67,7,Computada,La Paz,117,0,0,16,16,191,NO,MAS,False
7,6475251,2019-10-22 21:40:51,Bolivia,2,Muñecas,1,Chuma,,Chajlaya,Escuela Chajlaya,26765,100320202420240301,Presidente y Vicepresidente,207,5,0,8,1,145,3,14,0,0,176,7,8,Computada,La Paz,176,0,0,16,16,191,NO,MAS,False
8,6032792,2019-10-22 04:52:51,Bolivia,7,Guarayos,1,Ascención de Guarayos,,Ascención (Guarayos),Esc. Ascencion Central,78510,100320725181700019,Presidente y Vicepresidente,232,36,0,2,0,63,51,2,2,2,158,41,2,Computada,Santa Cruz,158,0,0,31,31,201,NO,MAS,False
9,7302459,2019-10-23 22:28:43,Bolivia,7,Guarayos,1,Ascención de Guarayos,,Ascención (Guarayos),Esc. Ascencion Central,78510,100320725181700019,Presidente y Vicepresidente,232,57,1,0,0,86,36,6,2,2,190,5,6,Computada,Santa Cruz,190,0,0,31,31,201,NO,MAS,False


In [0]:
 df_mesas_corregidas_detalle.to_csv('MesasCorregidas.csv')

In [33]:
#@title Mesa a buscar { run: "auto", form-width: "800px" }
mesa = "100320722341108805" #@param {type:"string"}
VER = "EDICIONES UNICAS" #@param ["TODOS", "EDICIONES UNICAS", "ULTIMA ACTA"]

df_mesa = df_actas_all[df_actas_all['CodigoMesa'] == mesa].copy()
if VER == "EDICIONES UNICAS":
  df_mesa['Duplicated'] = df_mesa.duplicated(subset=df_mesa.columns[~df_mesa.columns.isin(['Id',	'FechaHoraArchivo'])])
  display(df_mesa[df_mesa['Duplicated'] == False])

elif VER == "ULTIMA ACTA":
  display(df_actas_all[(df_actas_all['FechaHoraArchivo'] == acta_final_fecha) & (df_actas_all['CodigoMesa'] == mesa)])
else:
  display(df_mesa)

,Id,FechaHoraArchivo,Pais,NumeroDepartamento,Provincia,NumeroMunicipio,Municipio,Circunscripcion,Localidad,Recinto,NumeroMesa,CodigoMesa,Eleccion,Inscritos,CC,FPV,MTS,UCS,MAS,M21F,PDC,MNR,PanBol,Validos,Blancos,Nulos,Computada,Departamento,_ValidosCalculado,_Validos_menos_ValidosCalculado,_Validos_menos_ValidosCalculado_Abs,_Ausentismo_OEP,_Ausentismo_Calculado,_Votantes,_TotalAsistencia,_PartidoGanador,Duplicated
390148,6027265,2019-10-22 04:52:51,Bolivia,7,Andres Ibañez,1,Santa Cruz de La Sierra,,Santa Cruz de la Sierra,U.E. 19 de Marzo,73415,100320722341108805,Presidente y Vicepresidente,235,49,1,1,0,116,26,0,6,0,180,32,3,Computada,Santa Cruz,199,-19,19,20,1,234,NO,MAS,False
1658889,7296006,2019-10-23 22:28:43,Bolivia,7,Andres Ibañez,1,Santa Cruz de La Sierra,,Santa Cruz de la Sierra,U.E. 19 de Marzo,73415,100320722341108805,Presidente y Vicepresidente,235,49,0,2,0,127,17,13,1,0,209,1,5,Computada,Santa Cruz,209,0,0,20,20,215,NO,MAS,False


### Orden de Procesamiento de actas

In [34]:
df_llegada_actas = df_actas_all.drop_duplicates(subset=['CodigoMesa'], keep='first').copy()
df_llegada_actas.sort_values(by=["FechaHoraArchivo", "_PartidoGanador"], inplace=True)
print(len(df_llegada_actas))
df_llegada_actas.head()

34551


,Id,FechaHoraArchivo,Pais,NumeroDepartamento,Provincia,NumeroMunicipio,Municipio,Circunscripcion,Localidad,Recinto,NumeroMesa,CodigoMesa,Eleccion,Inscritos,CC,FPV,MTS,UCS,MAS,M21F,PDC,MNR,PanBol,Validos,Blancos,Nulos,Computada,Departamento,_ValidosCalculado,_Validos_menos_ValidosCalculado,_Validos_menos_ValidosCalculado_Abs,_Ausentismo_OEP,_Ausentismo_Calculado,_Votantes,_TotalAsistencia,_PartidoGanador
0,5637117,2019-10-20 22:29:52,Bolivia,1,Oropeza,1,Sucre,,Sucre,Salon Multifuncional America,10177,100320127940022503,Presidente y Vicepresidente,232,104,3,1,0,41,9,26,2,3,189,5,9,Computada,Chuquisaca,189,0,0,29,29,203,NO,CC
467,5637584,2019-10-20 22:29:52,Bolivia,5,Tomas Frias,1,Potosí,,Potosí,Unidad Educativa Juan Pablo II,50635,100320514940048414,Presidente y Vicepresidente,228,114,0,0,1,31,15,27,1,3,192,2,10,Computada,Potosí,192,0,0,24,24,204,NO,CC
468,5637585,2019-10-20 22:29:52,Bolivia,5,Tomas Frias,1,Potosí,,Potosí,Unidad Educativa Juan Pablo II,50636,100320514940048415,Presidente y Vicepresidente,232,113,0,2,2,41,11,30,1,1,201,4,11,Computada,Potosí,201,0,0,16,16,216,NO,CC
469,5637586,2019-10-20 22:29:52,Bolivia,5,Tomas Frias,1,Potosí,,Potosí,Unidad Educativa Juan Pablo II,50637,100320514940048416,Presidente y Vicepresidente,227,119,0,1,1,33,15,25,0,2,196,1,9,Computada,Potosí,196,0,0,21,21,206,NO,CC
470,5637587,2019-10-20 22:29:52,Bolivia,5,Tomas Frias,1,Potosí,,Potosí,Unidad Educativa Juan Pablo II,50638,100320514940048417,Presidente y Vicepresidente,238,107,2,5,2,45,11,32,1,2,207,3,8,Computada,Potosí,207,0,0,20,20,218,NO,CC


In [35]:
df_llegada_actas_resumen = df_llegada_actas.groupby(by=['FechaHoraArchivo', 'Departamento', '_PartidoGanador']).count()[['CodigoMesa']].reset_index()
df_llegada_actas_resumen.head()

,FechaHoraArchivo,Departamento,_PartidoGanador,CodigoMesa
0,2019-10-20 22:29:52,Chuquisaca,CC,14
1,2019-10-20 22:29:52,Cochabamba,CC,26
2,2019-10-20 22:29:52,Cochabamba,MAS,20
3,2019-10-20 22:29:52,La Paz,CC,40
4,2019-10-20 22:29:52,La Paz,MAS,79


In [36]:
configure_plotly_browser_state()
df_llegada_actas_mas_cc = df_llegada_actas_resumen[df_llegada_actas_resumen['_PartidoGanador'].isin(['MAS','CC', 'PDC'])]
fig = px.bar(df_llegada_actas_mas_cc, x="FechaHoraArchivo",  title="Orden de llegada de actas", color= "_PartidoGanador" , text="Departamento")
fig.show()

In [37]:
df_llegada_actas_resumen_votos = df_llegada_actas.groupby(by=['FechaHoraArchivo', '_PartidoGanador', 'Provincia']).sum()[['MAS', 'CC']].reset_index()
df_llegada_actas_resumen_votos.head()

,FechaHoraArchivo,_PartidoGanador,Provincia,MAS,CC
0,2019-10-20 22:29:52,CC,Andres Ibañez,2979,14996
1,2019-10-20 22:29:52,CC,Antonio Quijarro,24,56
2,2019-10-20 22:29:52,CC,Cercado,5871,16712
3,2019-10-20 22:29:52,CC,Chapare,140,222
4,2019-10-20 22:29:52,CC,Murillo,2156,4091


In [38]:
configure_plotly_browser_state()
fig = px.bar(df_llegada_actas_resumen_votos, x="FechaHoraArchivo", y="MAS",  title="Orden de llegada de actas - Cantidad de votos MAS", color= "Provincia" )
fig.show()

In [39]:
configure_plotly_browser_state()
fig = px.bar(df_llegada_actas_resumen_votos, x="FechaHoraArchivo", y="CC",  title="Orden de llegada de actas - Cantidad de votos CC", color= "Provincia" )
fig.show()